Tests the precision of the ONNX and ONNX-FP16 models compared to the base one

In [11]:
from run_benchmark import sentiment_analysis_batched
import pandas as pd
%load_ext line_profiler

import argparse
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from optimum.onnxruntime import ORTModelForSequenceClassification
import time

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [7]:
# Models
model_id = "SamLowe/roberta-base-go_emotions"

model_id_onnx = "SamLowe/roberta-base-go_emotions-onnx"
file_name_onnx = "onnx/model.onnx"

model_id_onnx_fp16 = "joaopn/roberta-base-go_emotions-onnx-fp16"
file_name_onnx_fp16 = "model.onnx"

df = pd.read_csv('data/random_sample_10k.csv.gz', compression='gzip')

In [13]:
field = 'body'
batch_size = 8
gpu_id = 0

model_id = "SamLowe/roberta-base-go_emotions"

model_id_onnx = "SamLowe/roberta-base-go_emotions-onnx"
file_name_onnx = "onnx/model.onnx"

model_id_onnx_fp16 = "joaopn/roberta-base-go_emotions-onnx-fp16"
file_name_onnx_fp16 = "model.onnx"

_,_,df_test1b = sentiment_analysis_batched(df, 'gpu', 'torch', model_id, batch_size, field, file_name = None, gpu_id=0)
_,_,df_test2b = sentiment_analysis_batched(df, 'gpu', 'onnx', model_id_onnx, batch_size, field, file_name = file_name_onnx, gpu_id=0)
_,_,df_test3b = sentiment_analysis_batched(df, 'gpu', 'onnx', model_id_onnx_fp16, batch_size, field, file_name = file_name_onnx_fp16, gpu_id=0)

(Batch size 8): 100%|██████████| 1250/1250 [00:15<00:00, 82.48it/s]
c:\Users\Joao\miniconda3\envs\bench\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The ONNX file onnx/model.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.
(Batch size 8): 100%|██████████| 1250/1250 [00:10<00:00, 123.53it/s]
c:\Users\Joao\miniconda3\envs\bench\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
(Batch size 8): 100%|██████████| 1250/1250 [00:04<00:00, 255.33it/s]


In [3]:
field = 'body'
batch_size = 8
%lprun -u 1 -f sentiment_analysis_batched sentiment_analysis_batched(model_id, df, field,batch_size, 0)

(Batch size 8): 100%|██████████| 1250/1250 [00:17<00:00, 69.53it/s]


Timer unit: 1 s

Total time: 18.7622 s
File: c:\Users\Joao\repo\gpu_benchmark_goemotions\run_benchmark.py
Function: sentiment_analysis_batched at line 56

Line #      Hits         Time  Per Hit   % Time  Line Contents
    56                                           def sentiment_analysis_batched(model_id, df, field_name, batch_size, gpu_id):
    57         1          0.3      0.3      1.4      tokenizer = AutoTokenizer.from_pretrained(model_id, legacy=False)
    58         1          0.4      0.4      2.2      model = AutoModelForSequenceClassification.from_pretrained(model_id)
    59         1          0.0      0.0      0.0      device = torch.device(f'cuda:{gpu_id}' if torch.cuda.is_available() else 'cpu')
    60         1          0.1      0.1      0.5      model.to(device)
    61                                           
    62         1          0.0      0.0      0.0      start_time = time.time()
    63         1          0.0      0.0      0.0      results = []
    64           

In [6]:
_,_,df3 = sentiment_analysis_batched(model_id, df, field,batch_size, 0)

(Batch size 8): 100%|██████████| 1250/1250 [00:16<00:00, 73.64it/s]


In [7]:
df3

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,index
0,0.001547,0.000389,0.002694,0.045760,0.147581,0.008183,0.033266,0.028019,0.002859,0.024921,...,0.000842,0.014535,0.000205,0.026043,0.000712,0.000755,0.002815,0.000929,0.621454,0
1,0.001154,0.000910,0.001121,0.008919,0.028587,0.003058,0.030400,0.004150,0.002700,0.003179,...,0.000338,0.010806,0.000190,0.017915,0.000353,0.000401,0.000934,0.000474,0.924112,1
2,0.001994,0.001221,0.002787,0.011870,0.020030,0.001793,0.256123,0.464083,0.001105,0.002640,...,0.000399,0.006044,0.000079,0.008397,0.000168,0.000372,0.001185,0.005477,0.500028,2
3,0.003837,0.001673,0.002111,0.011314,0.017059,0.001365,0.003035,0.004966,0.000893,0.002489,...,0.000172,0.002373,0.000197,0.004484,0.000212,0.000186,0.000936,0.000574,0.952363,3
4,0.001439,0.001808,0.001421,0.005099,0.020517,0.002736,0.409891,0.521960,0.002616,0.002120,...,0.001180,0.013635,0.000098,0.015158,0.000346,0.000803,0.001660,0.008087,0.287632,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,0.005196,0.002714,0.002323,0.035727,0.132416,0.037693,0.001585,0.000487,0.006046,0.047171,...,0.004128,0.020429,0.006217,0.036360,0.056457,0.001847,0.021115,0.000661,0.403003,9995
4,0.003018,0.004345,0.005228,0.008969,0.010784,0.001430,0.001478,0.001477,0.001170,0.002646,...,0.000870,0.001874,0.000868,0.006598,0.000678,0.000445,0.002911,0.002966,0.951456,9996
5,0.003253,0.001277,0.000587,0.005238,0.227706,0.000934,0.034716,0.002264,0.001344,0.004616,...,0.000468,0.010513,0.000466,0.119277,0.000887,0.000623,0.001027,0.001024,0.661385,9997
6,0.007561,0.891062,0.001462,0.015509,0.010253,0.000826,0.008041,0.008290,0.001444,0.014622,...,0.001107,0.003611,0.000415,0.035765,0.001419,0.001488,0.008558,0.008100,0.054480,9998


In [9]:
def sentiment_analysis_batched2(model_id, df, field_name, batch_size, gpu_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id, legacy=False)
    model = AutoModelForSequenceClassification.from_pretrained(model_id)
    device = torch.device(f'cuda:{gpu_id}' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    start_time = time.time()
    results = []

    # Precompute id2label mapping
    id2label = model.config.id2label

    for start_idx in tqdm(range(0, len(df), batch_size), desc=f"(Batch size {batch_size})"):
        end_idx = start_idx + batch_size
        texts = df[field_name].iloc[start_idx:end_idx].tolist()

        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.nn.functional.sigmoid(outputs.logits)  # Use sigmoid for multi-label classification

        # Collect predictions on GPU
        results.append(predictions)

    # Concatenate all results on GPU
    all_predictions = torch.cat(results, dim=0).cpu().numpy()

    # Convert to DataFrame
    results_df = pd.DataFrame(all_predictions, columns=[id2label[i] for i in range(all_predictions.shape[1])])
    results_df['index'] = range(len(df))

    elapsed_time = time.time() - start_time
    messages_per_second = len(df) / elapsed_time

    return elapsed_time, messages_per_second, results_df

In [5]:
def sentiment_analysis_batched_test(model_type, model_id, df, field_name, batch_size, gpu_id, file_name=None):

    if model_type == 'torch':
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForSequenceClassification.from_pretrained(model_id)
        device = torch.device(f'cuda:{gpu_id}' if torch.cuda.is_available() else 'cpu')
        model.to(device)

    elif model_type == 'onnx':
        model = ORTModelForSequenceClassification.from_pretrained(model_id, file_name=file_name, provider="CUDAExecutionProvider", provider_options={'device_id': gpu_id})
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        device = torch.device(f'cuda:{gpu_id}' if torch.cuda.is_available() else 'cpu')

    start_time = time.time()
    results = []

    # Precompute id2label mapping
    id2label = model.config.id2label

    for start_idx in tqdm(range(0, len(df), batch_size), desc=f"(Batch size {batch_size})"):
        end_idx = start_idx + batch_size
        texts = df[field_name].iloc[start_idx:end_idx].tolist()

        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.nn.functional.sigmoid(outputs.logits)  # Use sigmoid for multi-label classification

        # Collect predictions on GPU
        results.append(predictions)

    # Concatenate all results on GPU
    all_predictions = torch.cat(results, dim=0).cpu().numpy()

    # Convert to DataFrame
    predictions_df = pd.DataFrame(all_predictions, columns=[id2label[i] for i in range(all_predictions.shape[1])])

    # Add prediction columns to the original DataFrame
    combined_df = pd.concat([df.reset_index(drop=True), predictions_df], axis=1)

    elapsed_time = time.time() - start_time
    messages_per_second = len(df) / elapsed_time

    return elapsed_time, messages_per_second, combined_df


In [8]:
field = 'body'
batch_size = 8
gpu_id = 0

model_id = "SamLowe/roberta-base-go_emotions"

model_id_onnx = "SamLowe/roberta-base-go_emotions-onnx"
file_name_onnx = "onnx/model.onnx"

model_id_onnx_fp16 = "joaopn/roberta-base-go_emotions-onnx-fp16"
file_name_onnx_fp16 = "model.onnx"

_,_,df_test1 = sentiment_analysis_batched_test('torch', model_id, df, field,batch_size, 0)
_,_,df_test2 = sentiment_analysis_batched_test('onnx', model_id_onnx, df, field,batch_size, 0, file_name_onnx)
_,_,df_test3 = sentiment_analysis_batched_test('onnx', model_id_onnx_fp16, df, field,batch_size, 0, file_name_onnx_fp16)

(Batch size 8): 100%|██████████| 1250/1250 [00:15<00:00, 82.71it/s]
c:\Users\Joao\miniconda3\envs\bench\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The ONNX file onnx/model.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.
(Batch size 8): 100%|██████████| 1250/1250 [00:10<00:00, 122.08it/s]
c:\Users\Joao\miniconda3\envs\bench\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
(Batch size 8): 100%|██████████| 1250/1250 [00:04<00:00, 250.15it/s]


In [17]:
def sentiment_analysis_batched3(model_id, df, field_name, batch_size, gpu_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id, legacy=False)
    model = AutoModelForSequenceClassification.from_pretrained(model_id)
    device = torch.device(f'cuda:{gpu_id}' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    start_time = time.time()
    results = []

    # Precompute id2label mapping
    id2label = model.config.id2label

    for start_idx in tqdm(range(0, len(df), batch_size), desc=f"(Batch size {batch_size})"):
        end_idx = start_idx + batch_size
        texts = df[field_name].iloc[start_idx:end_idx].tolist()

        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.nn.functional.sigmoid(outputs.logits)  # Use sigmoid for multi-label classification

        # Collect predictions on GPU
        results.append(predictions)

    # Concatenate all results on GPU
    all_predictions = torch.cat(results, dim=0).cpu().numpy()

    # Convert to DataFrame
    predictions_df = pd.DataFrame(all_predictions, columns=[id2label[i] for i in range(all_predictions.shape[1])])

    # Add prediction columns to the original DataFrame
    combined_df = pd.concat([df.reset_index(drop=True), predictions_df], axis=1)

    elapsed_time = time.time() - start_time
    messages_per_second = len(df) / elapsed_time

    return elapsed_time, messages_per_second, combined_df

In [29]:
model_type = 'onnx'
%lprun -u 1 -f sentiment_analysis_batched_test sentiment_analysis_batched_test(model_type, model_id_onnx, df, field,batch_size, 0, file_name_onnx)

c:\Users\Joao\miniconda3\envs\bench\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The ONNX file onnx/model.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.
(Batch size 8): 100%|██████████| 1250/1250 [00:10<00:00, 114.41it/s]


Timer unit: 1 s

Total time: 13.0405 s
File: C:\Users\Joao\AppData\Local\Temp\ipykernel_19972\155550149.py
Function: sentiment_analysis_batched_test at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def sentiment_analysis_batched_test(model_type, model_id, df, field_name, batch_size, gpu_id, file_name=None):
     2                                           
     3         1          0.0      0.0      0.0      if model_type == 'torch':
     4                                                   tokenizer = AutoTokenizer.from_pretrained(model_id, legacy=False)
     5                                                   model = AutoModelForSequenceClassification.from_pretrained(model_id)
     6                                                   device = torch.device(f'cuda:{gpu_id}' if torch.cuda.is_available() else 'cpu')
     7                                                   model.to(device)
     8                      

In [16]:
%lprun -u 1 -f sentiment_analysis_batched2 sentiment_analysis_batched2('onnx', model_id_onnx, df, field,batch_size, 0, file_name_onnx)

(Batch size 8): 100%|██████████| 1250/1250 [00:17<00:00, 69.68it/s]


Timer unit: 1 s

Total time: 18.8815 s
File: C:\Users\Joao\AppData\Local\Temp\ipykernel_60264\3222582611.py
Function: sentiment_analysis_batched2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def sentiment_analysis_batched2(model_id, df, field_name, batch_size, gpu_id):
     2         1          0.4      0.4      2.0      tokenizer = AutoTokenizer.from_pretrained(model_id, legacy=False)
     3         1          0.5      0.5      2.8      model = AutoModelForSequenceClassification.from_pretrained(model_id)
     4         1          0.0      0.0      0.0      device = torch.device(f'cuda:{gpu_id}' if torch.cuda.is_available() else 'cpu')
     5         1          0.0      0.0      0.2      model.to(device)
     6                                           
     7         1          0.0      0.0      0.0      start_time = time.time()
     8         1          0.0      0.0      0.0      results = []
     9        

In [18]:
_,_,df5 = sentiment_analysis_batched3(model_id, df, field,batch_size, 0)

(Batch size 8): 100%|██████████| 1250/1250 [00:16<00:00, 76.77it/s]


In [12]:
_,_,df4 = sentiment_analysis_batched2(model_id, df, field,batch_size, 0)

(Batch size 8): 100%|██████████| 1250/1250 [00:15<00:00, 80.96it/s]


In [12]:
field = 'body'
%lprun -u 1 -f sentiment_analysis_batched sentiment_analysis_batched(model_id, df, field,batch_size, 0)

(batch size 8): 100%|██████████| 1250/1250 [00:35<00:00, 35.61it/s]


Timer unit: 1 s

Total time: 36.0961 s
File: c:\Users\Joao\repo\gpu_benchmark_goemotions\run_benchmark.py
Function: sentiment_analysis_batched at line 56

Line #      Hits         Time  Per Hit   % Time  Line Contents
    56                                           def sentiment_analysis_batched(model_id, df, field_name, batch_size, gpu_id):
    57         1          0.4      0.4      1.2      tokenizer = AutoTokenizer.from_pretrained(model_id, legacy=False)
    58         1          0.5      0.5      1.4      model = AutoModelForSequenceClassification.from_pretrained(model_id)
    59         1          0.0      0.0      0.1      model.to(f'cuda:{gpu_id}')
    60                                           
    61                                               # Function to classify emotions of multiple texts in batched mode and return scores
    62         1          0.0      0.0      0.0      def classify_texts(texts):
    63                                                   inputs = t

In [6]:
field = 'body'
batch_size = 8
file_name_onnx = "onnx/model.onnx"
gpu_id = 0
%lprun -u 1 -f sentiment_analysis_onnx_batched sentiment_analysis_onnx_batched(model_id_onnx, file_name_onnx, df, field, batch_size, gpu_id)

The ONNX file onnx/model.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.
(batch size 8): 100%|██████████| 1250/1250 [00:27<00:00, 45.76it/s]


Timer unit: 1 s

Total time: 30.3518 s
File: c:\Users\Joao\repo\gpu_benchmark_goemotions\run_benchmark.py
Function: sentiment_analysis_onnx_batched at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def sentiment_analysis_onnx_batched(model_id, file_name, df, field_name, batch_size, gpu_id):
    10                                               
    11         1          2.8      2.8      9.2      model = ORTModelForSequenceClassification.from_pretrained(model_id, file_name=file_name, provider="CUDAExecutionProvider", provider_options={'device_id': gpu_id})
    12         1          0.2      0.2      0.7      tokenizer = AutoTokenizer.from_pretrained(model_id)
    13                                           
    14                                               # Function to classify emotions of multiple texts in batched mode and return scores
    15         1          0.0      0.0      0.0      def classify_texts(t

In [8]:
field = 'body'
batch_size = 8
file_name_onnx = "onnx/model.onnx"
gpu_id = 0
_,_,df2 = sentiment_analysis_onnx_batched(model_id_onnx, file_name_onnx, df, field, batch_size, gpu_id)

The ONNX file onnx/model.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.
(batch size 8): 100%|██████████| 1250/1250 [00:28<00:00, 44.05it/s]


In [6]:
import argparse
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from optimum.onnxruntime import ORTModelForSequenceClassification
import time


def analyze_single_input_onnx(text):

    model_id = "SamLowe/roberta-base-go_emotions-onnx"
    file_name = "onnx/model.onnx"
    gpu_id = 0

    model = ORTModelForSequenceClassification.from_pretrained(model_id, file_name=file_name, provider="CUDAExecutionProvider", provider_options={'device_id': gpu_id})
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    
    probabilities = torch.sigmoid(outputs.logits)
    labels = model.config.id2label  # Adjust if necessary
    result = {labels[i]: prob_item.item() for i, prob_item in enumerate(probabilities.squeeze())}

    return result

In [7]:
text = "If you're targeting the JVM and ecosystem, you might prefer a language designed from the ground up to play within that ecosystem, rather than, say, something from the posix/C ecosystem ported to the JVM."
analyze_single_input_onnx(text)

The ONNX file onnx/model.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.


{'admiration': 0.0011540877167135477,
 'amusement': 0.0009100200841203332,
 'anger': 0.0011203810572624207,
 'annoyance': 0.008914163336157799,
 'approval': 0.0285797081887722,
 'caring': 0.003058207919821143,
 'confusion': 0.03042701445519924,
 'curiosity': 0.004150078631937504,
 'desire': 0.0027006366290152073,
 'disappointment': 0.0031804495956748724,
 'disapproval': 0.018121996894478798,
 'disgust': 0.0012776002986356616,
 'embarrassment': 0.00030603265622630715,
 'excitement': 0.00048009015154093504,
 'fear': 0.0007079296628944576,
 'gratitude': 0.0009287563734687865,
 'grief': 0.0002476806694176048,
 'joy': 0.0006493583787232637,
 'love': 0.0008363121887668967,
 'nervousness': 0.0003379562112968415,
 'optimism': 0.010804506950080395,
 'pride': 0.00018951600941363722,
 'realization': 0.0179132167249918,
 'relief': 0.00035339107853360474,
 'remorse': 0.0004010939155705273,
 'sadness': 0.0009339235257357359,
 'surprise': 0.0004745300393551588,
 'neutral': 0.9240930676460266}